In [100]:
import pandas as pd
from IPython.display import HTML, display
from os.path import join as pjoin
import csv
import pickle


#m = pd.read_csv("Airbnb_Texas_Rentals.csv",encoding = 'utf8')

In [101]:
#we found words like '\\n' in the dataset so we cleaned it
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\t',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\r',  ' ', regex=True)

In [102]:
#create tsv files,we droped the first column that was a prroblem
#m = m.drop(['Unnamed: 0'], axis=1)

In [103]:
#we create the vocabulary of preprocessed documents,but we don't modify the documents because we''l use them in search engine
import nltk
import csv
import re
import os
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+|\$')
ps = SnowballStemmer('english')

dic = {}
my_path = os.path.dirname(os.path.realpath('__file__'))

for i in range(18259):
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(i) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    l1 = tsvreader[0][4]
    l2 = tsvreader[0][7]
    l = l1+ ' ' +l2
    l = l.lower()
    l = tokenizer.tokenize(l)
    for r in l :
        if not r in stop_words:
            sr = ps.stem(r)
            if not  sr in dic:
                dic[sr] = [i]
            else:
                dic[sr]+=[i]
                


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vamsigunturi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
# create dictionary of words
dic_word = {}
dic_assoc = {}
c = 0
for key in dic:
    dic_word[key] = c
    dic_assoc[c] = dic[key]
    c += 1

# Persisting the indexes created using the pickle library
with open(os.path.join(my_path, "word_index.pkl"), "wb") as word_index:
    pickle.dump(dic_word, word_index)
    word_index.close()
    
with open(os.path.join(my_path, "index_word.pkl"), "wb") as index_word:
    pickle.dump(dic_assoc, index_word)
    index_word.close()        
    
    

In [105]:
# create dictionary of documents
# @TODO: Need to remove this dictionary if not used
dic_doc={}
for i in range(18259):
    dic_doc[r'docu_hw3\doc_'+str(i)+'.tsv']= i

In [106]:
word = input('Enter a search query: ')


# Retrieving persisted information

with open(os.path.join(my_path, "word_index.pkl"), "rb") as word_index:
    dic_word_persist = pickle.load(word_index)
    word_index.close()
    
with open(os.path.join(my_path, "index_word.pkl"), "rb") as index_word:
    dic_assoc_persist = pickle.load(index_word)
    index_word.close() 
    
    
def clean_input(w):
    w_list = []
    w = w.lower()
    w = tokenizer.tokenize(w)
    # Check if we need to do any other preprocessing to improve the efficiency of search results
    
    for r in w :
        if not r in stop_words:
            sr = ps.stem(r)
            if not  sr in w_list:
                w_list.append(sr)
    return w_list

word_list = clean_input(word)

print("Cleaned word: ", word_list)

list_doc_list = []

for w in word_list:
        doc_list = []
        
        if w in dic_word: 
            doc_list = dic_assoc_persist[dic_word_persist[w]]
        list_doc_list.append(doc_list)  
        
list_intersect = list_doc_list[0]

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

        
for docList in list_doc_list:
    list_intersect = intersection(list_intersect, docList)
    

results = []

        
for doc in list_intersect:
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    
    results.append([title, description, city, url])


    
if(len(results)):
    print('Found ' + str(len(results))  + ' matching reviews to your query')
    display(HTML('<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results)
)))
else:
    display(HTML('<h1>No results found. Please try a different query</h1>'))

    

Enter a search query: beautiful house
Cleaned word:  ['beauti', 'hous']
Found 920 matching reviews to your query


Title,Description,City,URL
Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. Amazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. Featuring an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more European inspired kitchen and “top of the line” decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX
Friendly Private Room in َQuiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX
A Cozy Home with a Beautiful Nature Views,"This home is on the North Side of San Antonio, 3 minutes away from a great golfing course and a JW Marriott Resort and Spa. This is a great stopping place for a family and comes with all amenities from cable, internet to a laundry room and 2 bathrooms. This house is against a green belt so you can enjoy the back yard and nature.This area is safe and my neighbors are also great! Hope you enjoy your stay and more so enjoy San Antonio!",San Antonio,https://www.airbnb.com/rooms/19190311?location=Cibolo%2C%20TX
Monroe Manor,Come relax on the water in our beautiful beach house style river home. You'll love our shiplap walls but the view is even better!,Houston,https://www.airbnb.com/rooms/19273801?location=Channelview%2C%20TX
Little Blanco,"Little Blanco is a gorgeous vacation rental house located in Concan, Texas. It is in the beautiful 18 hole golf course in Concan. It is also on one of the highest points of the Golf Course. It has over 4342 sq. ft. of livable space for you and your guests. It has 4 bedrooms and 4 bathrooms and sleeps 28 people. It is a single level home.",Concan,https://www.airbnb.com/rooms/17808720?location=Concan%2C%20TX
Rent a Home! Built 2015! Corner lot!,"Rent this beautiful home built in 2015! It's in the new section of the subdivision. The house has 4 bedrooms and 2 baths. It is a one and a half story home with 2300 sq ft in total. The house is on a corner lot located in Conroe, TX, about 10 minutes drive to The Woodlands or Lake Conroe. Please let me know if you are interested. Thanks for looking!",Conroe,https://www.airbnb.com/rooms/16655071?location=Cleveland%2C%20TX
Game Day Getaway 10min from Stadium,"Beautiful 3/2.5 house located just 1 minute from Wellborn Road (which gives you a straight shot to Kyle Field). The house sleeps 6, but can easily accommodate more if willing to sleep on couch. It's a perfect place for Texas A&M rental.",College Station,https://www.airbnb.com/rooms/8499971?location=College%20Station%2C%20TX
Spacious home near Sea World and Lackland AFB,"Enjoy three large bedrooms and a loft. Entire house is for rent (3160 sq ft). Sleeps 8 comfortable. One king size bed, one queen bed, one double bed, and day bed (two twin beds). Full laundry. Near Air Force Base, Sea World (3 mi) and Six Flags and Riverwalk. Enjoy a beautiful living room, kitchen, pool table, and patio. Plenty of room for a big family or group. Fresh towels and linens. Nearby shopping, restaurants, and grocery store. Fully furnished kitchen. Safe and quiet neighborhood.",San Antonio,https://www.airbnb.com/rooms/16417007?location=Castroville%2C%20TX
MODERN HOME with SWIMMING POOL - The Woodlands,"This family home is located in Shenandoah,towards the outskirts of The Woodlands.The Woodlands is unique in its beauty, diversity, convenience and comfort.You can experience all of its beauty on your own in the comfort of your own personal vacation retreat.The house may look small from the outside, but ma